<a href="https://colab.research.google.com/github/mrudulagavas/All-About-AI-ML/blob/main/Gradient_Boost_Decision_Tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import numpy as np
from collections import Counter

In [8]:
import numpy as np
from collections import Counter

class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None,*,value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

    def is_leaf_node(self):
        return self.value is not None

Decision Tree Class

In [9]:
class DecisionTree:
    def __init__(self, min_samples_split=2, max_depth=100, n_features=None):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.n_features = n_features
        self.root = None

    def fit(self, X, y):
        self.n_features = X.shape[1] if not self.n_features else min(X.shape[1], self.n_features)
        self.root = self._grow_tree(X, y)

    class Node:
        def __init__(self, feature=None, threshold=None, left=None, right=None, value=None):
            self.feature = feature
            self.threshold = threshold
            self.left = left
            self.right = right
            self.value = value

        def is_leaf_node(self):
            return self.value is not None

    def _grow_tree(self, X, y, depth=0):
        n_samples, n_feats = X.shape

        if depth >= self.max_depth or n_samples < self.min_samples_split:
            leaf_value = self._mean_value(y)
            return self.Node(value=leaf_value)

        feat_idxs = np.random.choice(n_feats, self.n_features, replace=False)
        best_feature, best_thresh = self._best_split(X, y, feat_idxs)

        if best_feature is None:
            return self.Node(value=self._mean_value(y))

        left_idxs, right_idxs = self._split(X[:, best_feature], best_thresh)
        left = self._grow_tree(X[left_idxs], y[left_idxs], depth + 1)
        right = self._grow_tree(X[right_idxs], y[right_idxs], depth + 1)
        return self.Node(best_feature, best_thresh, left, right)

    def _best_split(self, X, y, feat_idxs):
        best_var_red = -1
        split_idx, split_thresh = None, None

        for feat_idx in feat_idxs:
            X_column = X[:, feat_idx]
            thresholds = np.unique(X_column)

            for threshold in thresholds:
                left_idxs, right_idxs = self._split(X_column, threshold)
                if len(left_idxs) == 0 or len(right_idxs) == 0:
                    continue

                var_red = self._variance_reduction(y, left_idxs, right_idxs)
                if var_red > best_var_red:
                    best_var_red = var_red
                    split_idx = feat_idx
                    split_thresh = threshold

        return split_idx, split_thresh

    def _variance_reduction(self, y, left_idxs, right_idxs):
        var_total = np.var(y)
        var_left = np.var(y[left_idxs])
        var_right = np.var(y[right_idxs])
        n, n_l, n_r = len(y), len(left_idxs), len(right_idxs)

        weighted_var = (n_l / n) * var_left + (n_r / n) * var_right
        return var_total - weighted_var

    def _split(self, X_column, threshold):
        left_idxs = np.argwhere(X_column <= threshold).flatten()
        right_idxs = np.argwhere(X_column > threshold).flatten()
        return left_idxs, right_idxs

    def _mean_value(self, y):
        return np.mean(y)

    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])

    def _traverse_tree(self, x, node):
        if node.is_leaf_node():
            return node.value

        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)


Gradient Boost Class

In [10]:
class GradientBoostedTrees:
    def __init__(self, n_estimators=100, learning_rate=0.1, max_depth=3):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.trees = []

    def _sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def fit(self, X, y):
        # Initialize with 0 prediction (logit = 0 => prob = 0.5)
        self.F0 = np.full(y.shape, 0.0)
        self.trees = []
        Fm = self.F0.copy()

        for _ in range(self.n_estimators):
            # Compute pseudo-residuals (negative gradient of log-loss)
            p = self._sigmoid(Fm)
            residuals = y - p

            # Fit tree to residuals
            tree = DecisionTree(max_depth=self.max_depth)
            tree.fit(X, residuals)
            prediction = tree.predict(X)

            # Update model with scaled tree prediction
            Fm += self.learning_rate * prediction
            self.trees.append(tree)

    def predict_proba(self, X):
        Fm = np.full((X.shape[0],), 0.0)
        for tree in self.trees:
            Fm += self.learning_rate * tree.predict(X)
        return self._sigmoid(Fm)

    def predict(self, X):
        proba = self.predict_proba(X)
        return (proba >= 0.5).astype(int)


In [11]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

In [12]:
# Load data
data = load_breast_cancer()
X, y = data.data, data.target

In [13]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
# Train GBDT
gbdt = GradientBoostedTrees(n_estimators=10, learning_rate=0.1, max_depth=3)
gbdt.fit(X_train, y_train)

In [15]:
# Evaluate
preds = gbdt.predict(X_test)
accuracy = np.mean(preds == y_test)
print("Gradient Boosted Trees Accuracy:", accuracy)

Gradient Boosted Trees Accuracy: 0.9473684210526315
